# RAG sur les description de produit Amazon

In [ ]:
from langchain_core.messages import AIMessage
from langchain.vectorstores import FAISS
from langchain_ollama import ChatOllama
from langchain_ollama.embeddings import OllamaEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import json

import time

In [2]:
path_desc ="sources\\meta.jsonl"

### Prétraitement et segmentation des textes


In [ ]:
descriptions = []
with open(path_desc, "r", encoding="utf-8") as f:
    for line in f:
        record = json.loads(line)
        description = record.get("description", "")
        descriptions.extend(description)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=128
    #diminuer l'overlap ne change pas grand chose au temps de chargemnt de l'embedding
)
#split descriptions in chunks
all_chunks = []

#subset bc it's too big to index, it takes too much time more than 10 min
subset_desc = descriptions[:300]
# for description in descriptions:
for description in subset_desc :
    # print(description)
    chunks = text_splitter.split_text(description)
    all_chunks.extend(chunks)

print(f"Nb total de desc : {len(descriptions)}")
print(f"Nb total de morceaux : {len(all_chunks)}")

# print(all_chunks[:2])
# print(type(all_chunks))

Nb total de desc : 2106
Nb total de morceaux : 327


### Index vectoriel

init model, embedding process

In [ ]:
embed = OllamaEmbeddings(model="llama3.2")

In [ ]:
start = time.time()
vectors = embed.embed_documents(all_chunks)
end = time.time()

print(f"embedding process time : {end - start} s")

embedding process time : 392.9859561920166 s


In [ ]:
# print(vectors[0])

### Vectorial DB & retrieval system

In [ ]:
text_embedding_pairs = list(zip(all_chunks, vectors))

vectorstore = FAISS.from_embeddings(text_embedding_pairs, embed)

retriever = vectorstore.as_retriever()

In [ ]:
#find documents alike
query = "peux-tu me donner une liste de coques pour un téléphone samsung s6 ?"
query_iphone= "peux tu me donner des informations sur la coque de téléphone dandycase"
queryOneplus="Tell me about OnePlus 6T"
querycase="Tell me about the soft TPU phone Case"
querysportband="Tell me about  Apple Watch Sport Band"

retrieved_docs = retriever.get_relevant_documents(query)

#combine retrieved content
documents = "\n\n".join([doc.page_content for doc in retrieved_docs])

llm = ChatOllama(
    model="llama3.2",
    temperature=0
)
 #temp proche de 0 = modèle prend les réponses avec la probabilité la plus haute, proche de 1 == moins déterministe

prompt = "Vous êtes un assistant intelligent qui répond aux questions des clients de la boutique en ligne en utilisant uniquement les informations contenues dans les documents pertinents fournisles voici :{documents}. dans ce contexte, le mot \"case\" se traduit par \"coque de téléphone\" en fran_ais.Basé uniquement sur ces documents, répondez précisément et de manière concise à la question suivante :"

messages = [
    (
        "system",
        prompt,
    ),
    (querysportband),
]
ai_msg = llm.invoke(messages)

In [ ]:
print(ai_msg.content)

Je n'ai trouvé aucune information spécifique sur un "Apple Watch Sport Band" dans les documents fournis. Cependant, je peux vous fournir des informations générales sur les bandes de l'Apple Watch.

Les bandes de l'Apple Watch sont conçues pour s'accrocher à la montre et offrir une protection supplémentaire contre les chocs et les dégâts. Elles sont disponibles en différentes couleurs, matériaux et styles

Si vous cherchez des informations spécifiques sur un modèle particulier de bande Apple Watch, je vous recommande de vérifier le site officiel d'Apple ou les documents fournis pour obtenir plus de détails.


In [ ]:
#tests sur la température
llm = ChatOllama(
    model="llama3.2",
    temperature=0.99,
    top_k=100,
    top_p=0.95
    #temp proche de 0 = modèle prend les réponses avec la probabilité la plus haute, proche de 1 == moins déterministe
)

messages = [
    (
        "system",
        prompt,
    ),
    (querysportband),
]
ai_msg = llm.invoke(messages)
print(ai_msg.content)


Désolé, je n'ai pas trouvé d'informations spécifiques sur un produit appelé "Apple Watch Sport Band". Cependant, j'ai trouvé des informations sur les bandes pour Apple Watch :
Les bandes de Apple Watch sont une gamme de accessoires conçus pour être compatibles avec l'horloge Apple Watch. Les bandes Sport de Apple Watch sont spécifiquement conçues pour les exercices et les activités sportives, offrant une confortabilité et une flexibilité exceptionnelle.

Voici quelques caractéristiques des bandes Sport de Apple Watch :

* Matériau : Cuir ou Nylone
* Couleurs : Variées (par exemple, blanc, noir, rose)
* Dimensions : 22mm, 38mm, 42mm et 44mm pour les modèles Apple Watch
* Poids : Legger et confortable
* Finitions : Plus de finitions disponibles dans le portail d'Apple ou dans certains retailers autorisés

Les bandes Sport de Apple Watch sont conçues pour être détachable, ce qui signifie qu'il est facile de les remplacer ou de les changer selon vos préférences. Elles sont également résist

In [ ]:
print(ai_msg.response_metadata)

{'model': 'llama3.2', 'created_at': '2024-12-08T00:25:43.2841438Z', 'done': True, 'done_reason': 'stop', 'total_duration': 42242257200, 'load_duration': 22925600, 'prompt_eval_count': 100, 'prompt_eval_duration': 3154000000, 'eval_count': 312, 'eval_duration': 39060000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}


In [13]:
# print(retrieved_docs)

check the existence of response content in retrieved docs

In [ ]:
kword = "One"
keyword = keyword.lower()
filtered_docs = [doc for doc in retrieved_docs if keyword in doc.page_content.lower()]

# print docs found
if filtered_docs:
    print(f"Documents contenant '{keyword}' :")
    for i, doc in enumerate(filtered_docs, 1):
        print(f"\nDocument {i}:\n{doc.page_content}")
else:
    print(f"Aucun document trouvé contenant '{keyword}'.")

Documents contenant 'one' :

Document 1:
your phone in your hand. About us. Yonds Queen is committed to creating the most fashionable, cutting-edge products and the best quality products to bring you a pleasant shopping experience in your leisure life. At the same time, our goal is to provide the highest standard of customer service to each and every one of our customers. If you have any questions, you can contact us promptly via email. Package contents. 1 * Cute soft TPU Case

Document 2:
Sport Band Compatible with Apple Watch Bands 38mm 40mm 41mm 42mm 44mm 45mm for Women Men,Floral Silicone Printed Fadeless Pattern Replacement Strap Band for iWatch Series 7,Series 3, Series 5,Series 6,Series 4
